In [8]:
# 코드 작성일 : 24.7.26
import os

if 'original_dir' not in globals() :
    original_dir = os.getcwd()
    original_dir = os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(original_dir))))
    os.chdir(original_dir)
exec(open('setup/default.py').read())

In [9]:
menu_w_sold_out = pd.read_csv(os.getcwd()+'/data/preproc/main/menu_w_soldout.csv')

In [10]:
# 클렌징 
def filter_text(text):
    # '>'가 있는 경우 '>' 뒤의 텍스트 추출
    if '>' in text:
        text = text.replace('>,', '>') # 만약 <~>,이런식으로 붙어있으면 변경해주기
        text = text.split('>')[-1]
    # ','가 있는 경우 ',' 앞의 텍스트 추출
    if ',' in text:
        text = text.split(',')[0]
    # 특수문자 이후의 텍스트 제거
    text = re.sub(r'[(*&].*$', '', text).strip()
    return text

menu_list = pd.read_csv(original_dir+'/data/feature_lab/menu_map/entity/spaced_menu.csv')

# Replacement rules
replacements = {
    "돈가스": "돈까스",
    "치킨가스": "치킨까스",
    "자장": "짜장",
    "코든블루": "코돈브루",
    "코든브루": "코돈브루",
    "코돈블루": "코돈브루",
    "플래이트": "",
    "플레이트": "",
    "플래터": "",
    "플레터": "",
    "정식": "",
    "소시지": "소세지",
    "돈유": "돈육",
    "함바그" : "함박",
    "커틀렛" : "커틀릿",
    "오무" : "오므",
    "뽀모로" : "뽀모도로"
}

def replace_text(text, replacements):
    for key, value in replacements.items():
        text = text.replace(key, value)
    # Remove digits
    text = ''.join([i for i in text if not i.isdigit()])
    return text


# 데이터프레임에 함수 적용
menu_w_sold_out['course'] = menu_w_sold_out['course'].apply(lambda x: replace_text(x, replacements))
menu_w_sold_out['first_menu'] = menu_w_sold_out['course'].apply(filter_text)

In [11]:
first_menu_list = menu_w_sold_out.groupby(['first_menu'],dropna=False).size().to_frame('size').reset_index().sort_values(by='size',ascending=False)

In [12]:
first_menu_list

,first_menu,size
317,뼈없는감자탕,21
94,닭곰탕,18
606,콩비지찌개,17
199,마파두부덮밥,16
145,돼지국밥,16
...,...,...
253,미트소스스파게티,1
251,미트볼로제스파게티,1
250,미니우동,1
249,미나리불고기,1


In [13]:
#!pip3 install konlpy

In [32]:
high_cat = pd.read_csv(original_dir+'/data/feature_lab/menu_map/entity/group/high_cat.csv').drop_duplicates()['high_cat'].tolist()
med_cat = pd.read_csv(original_dir+'/data/feature_lab/menu_map/entity/group/med_cat.csv').drop_duplicates()['med_cat'].tolist()
ingredient = pd.read_csv(original_dir+'/data/feature_lab/menu_map/entity/group/ingredient.csv').drop_duplicates()['ingredient'].tolist()

In [34]:
custom_nouns = high_cat+med_cat+ingredient
with open(original_dir+'/data/feature_lab/menu_map/entity/group/user_dic.txt', 'w', encoding='utf-8') as f:
    for noun in custom_nouns:
        f.write(f"{noun}\tNNG\n")


In [36]:
from konlpy.tag import Komoran

# 명사 리스트
custom_nouns = ['구이', '국', '국수', '개장', '까스', '나시고랭', '나베']

# 사용자 정의 사전 파일 작성
with open('user_dic.txt', 'w', encoding='utf-8') as f:
    for noun in custom_nouns:
        f.write(f"{noun}\tNNG\n")

# Komoran 객체 생성 시 사용자 정의 사전 로드
komoran = Komoran(userdic='user_dic.txt')

# 테스트 문장
text = "오늘 점심으로 나시고랭과 국수를 먹었다."
print(komoran.morphs(text))

['오늘', '점심', '으로', '나시고랭', '과', '국수', '를', '먹', '었', '다', '.']


In [42]:
from konlpy.tag import Komoran
Komoran = Komoran(userdic=original_dir+'/data/feature_lab/menu_map/entity/group/user_dic.txt')
text1 = '일식부타야끼우동'
text2 = '경양식미트볼오므라이스'
print(Komoran.morphs(text1))
print(Komoran.morphs(text2))



['일식', '부타', '야끼', '우동']
['경', '양식', '미트볼', '오므', '라이스']


In [73]:
text3 = ''
#print(Komoran.morphs(text3))
print(Komoran.nouns('중화풍 해물 잡채 덮 밥'))

['중', '화풍', '해물', '잡채', '덮', '밥']


In [40]:
original_dir+'/data/feature_lab/menu_map/entity/group/user_dic.txt'

'/Users/al01989093/Desktop/haeyoon_private/cafeteria/data/feature_lab/menu_map/entity/group/user_dic.txt'